### Fire Weather Prep
#### The goal of this notebook is to leverage RAWS data to generate fire weather inputs for LANDIS across the landscape. The notebook imports from FireWeatherIndex_Calculations.py, a script originially written by Louise Loudermilk to convert time series of daily data into the LANDIS input files required by the Dynamic Fire and Fuels System extension. Here, we modify this script and set up a work flow to process RAWS data by year.

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import os, datetime
from scipy import stats
from FireWeatherIndex_Calculations import DynamicFireWeatherCalculations
DFFSDir = 'C:/Research/Dinkey_Creek/DFFS_Data/FireWeather/'

#### I chose to represent fire weather using RAWS data from each of the three fire regions represented in our simulation. The fire regions were defiend by Shuang et al., 2015, and were defined using elevation bins. FR1 is the lowest elevation region, and RAWS data from Batterson California were used to generate the fire weather (elvation = 963 m). FR2 is the middle elevation region, with fireweather derived from the Dinkey California station (elevation = 1727 m). FR3 is the high elevation region, and I chose the High Sierra California RAWS station (elevation = 2256 m). All stations are fairly proximal to one another, and roughly represent the middle elevation of their respective fire regions.

#### RAWS data fills bad values with a mask (I chose -9999). That means we need to remove those rows before the fireweather prep script can operate on the downloaded data.

#### The current state of Louise's script needs some specific data formatting.. since some of the RAWS data starts or ends in the middle of  a year, I clipped the inputs to start in 2002 and end in 2014.. that way only complete years (mostly just that the data start on the 1st and end on the 365th day of year) are used in the analysis -- I Think this is only a problem because of the way the script assesses season, and think it might be worth modernizing the script to leverage Pandas and timestamps...

In [39]:
# Function to clean up teh mask values and data types from the RAWS outputs. Note, here the RAWS ASCII Downloads
# have to be manually edited to conform to the sample input from Louise's script in order to run.
def cleanRaws(formattedRawsData, maskValue, yearStart, yearStop, cleanedRawsData):
    rawsdata = pd.read_csv(formattedRawsData)
    rawsdata[rawsdata == maskValue] = np.nan
    rawsdata = rawsdata.dropna()
    rawsdata.Juliandate = rawsdata.Juliandate.astype(int)
    rawsdata.windazimuth = rawsdata.windazimuth.astype(int)
    rawsdata.relativehumidity = rawsdata.relativehumidity.astype(int)
    rawsdata = rawsdata[rawsdata.year > yearStart]
    rawsdata = rawsdata[rawsdata.year < yearStop]

    rawsdata.to_csv(cleanedRawsData, index = False)
    


In [44]:
# Clean up RAWS raw data and generate FW inputs for the three fire regions

cleanRaws(DFFSDir + 'BATTERSON_RAWS_FORMATTED.csv',-9999, 
          2001, 2014, DFFSDir + 'FR1_RawsWeather.csv')
cleanRaws(DFFSDir + 'DINKEY_RAWS_FORMATTED.csv',-9999,  
          2001, 2014, DFFSDir + 'FR2_RawsWeather.csv')
cleanRaws(DFFSDir + 'HIGH_SIERRA_RAWS_FORMATTED.csv',-9999,  
          2001, 2014, DFFSDir + 'FR3_RawsWeather.csv')


In [54]:
# Run Louise's script on the three fire regions input RAWS generated previously.

fireRegions = ['FR1','FR2','FR3']
for region in fireRegions:
    p = DynamicFireWeatherCalculations(DFFSDir + region +'_RawsWeather.csv',
                                       DFFSDir + 'sample_startingvalues_py.csv',
                                       DFFSDir +  region +'_FireWeather.csv', region)
    p.run()
FR1 = pd.read_csv(DFFSDir + 'FR1_FireWeather.csv')
FR2 = pd.read_csv(DFFSDir + 'FR2_FireWeather.csv')
FR3 = pd.read_csv(DFFSDir + 'FR3_FireWeather.csv')

DinkeyFW = pd.concat([FR1, FR2, FR3])
DinkeyFW.to_csv(DFFSDir + 'DinkeyFW.csv', index = False)

No. days being evaluated  4293

Writing output file for LANDIS-II  . . . 

 no. of daily values (rows) output to file =  1880
No. days being evaluated  4360

Writing output file for LANDIS-II  . . . 

 no. of daily values (rows) output to file =  1219
No. days being evaluated  4356

Writing output file for LANDIS-II  . . . 

 no. of daily values (rows) output to file =  2031


'C:/Research/Dinkey_Creek/DFFS_Data/FireWeather/DINKEY_RAWS_FORMATTED.csv'